In [1]:
# evaluate a persistence forecast model
from pandas import read_csv
import datetime
from pandas import DataFrame, Series
from pandas import concat, merge
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate, savetxt, unique, array, subtract
from statistics import mean


In [2]:
# make a prediction using data from a single meter
# load dataset
dataset = read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python') 
meter_ids = unique(dataset['MeterNo'])
print(meter_ids)
df2 = DataFrame(index =['Date'], columns=['prediction', 'actual', 'error','mse'])

[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269
 270 271 272 273 274 275 301 302 303 304 305 306 307 308 309 310 311 312
 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330
 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348
 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366
 367 368 369 370 371 372 373 374 375 401 402 403 404 405 406 407 408 409
 410 411 412 413 414 415 416 417 418 419 420 421 42

In [3]:
for m in meter_ids:
    series = read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python') 
    series.columns =['MeterNo', 'WeekNo', 'kWh']
    series = series.set_index(['WeekNo'])
    series = series.loc[series['MeterNo'] == m]
    series.drop(series.columns[[0]], axis = 1, inplace = True)
    
    # create lagged dataset
    values = DataFrame(series.values)
    dataframe = concat([values.shift(1), values], axis=1)
    dataframe.columns = ['t', 't+1']
    dataframe = dataframe.dropna()
    #print(dataframe.head(5))
    # split into train and test sets
    X = dataframe.values
    train_size = int(len(X) * 0.66)
    train, test = X[1:train_size], X[train_size:]
    train_X, train_y = train[:,0], train[:,1]
    test_X, test_y = test[:,0], test[:,1]
    # persistence model
    def model_persistence(x):
        return x
    # walk-forward validation
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
        
    # calculate actual Error
    prediction = mean(predictions)
    actual = mean(test_y)
    mse = mean_squared_error(test_y, predictions)
    rmse = sqrt(mean_squared_error(test_y, predictions))
    error = mean(subtract(test_y, predictions))
    
    test_y = DataFrame(test_y, columns = ['kWh'])
    predictions = DataFrame(predictions, columns = ['kWh']) 
        
    #print('MeterNo %s - Prediction: %.3f - Actual: %.3f - Error: %.3f - Test MSE: %.3f - Test RMSE: %.3f' % (m, prediction, actual, error, mse, rmse))
    df1 = merge(predictions, test_y, left_index=True, right_index=True)
    df1.columns =['prediction', 'actual']
    df1['error'] = df1['prediction'] - df1['actual']
    df1['mse'] = round(df1['error'] ** 2,2)
    df1['rmse'] = abs(df1['error'])
    #print(df1)
    df2 = df2.append(df1)
    

In [4]:
# Each meter per week
df2 = df2.dropna()
print(df2)

    prediction  actual  error       mse   rmse
0        139.0   118.0   21.0     441.0   21.0
1        118.0   117.0    1.0       1.0    1.0
2        117.0   114.0    3.0       9.0    3.0
3        114.0    92.0   22.0     484.0   22.0
4         92.0    85.0    7.0      49.0    7.0
..         ...     ...    ...       ...    ...
28       368.0   653.0 -285.0   81225.0  285.0
29       653.0  1114.0 -461.0  212521.0  461.0
30      1114.0  1721.0 -607.0  368449.0  607.0
31      1721.0  1797.0  -76.0    5776.0   76.0
32      1797.0  1719.0   78.0    6084.0   78.0

[9405 rows x 5 columns]


In [5]:
# Summary of each meter for each week
def Summary(x):
    return round(Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()]),2)
df2.apply(Summary)

,prediction,actual,error,mse,rmse
min,-1821.00,-1821.0,-1205.00,0.000000e+00,0.00
max,2152.00,2152.0,1435.00,2.059225e+06,1435.00
mean,349.89,356.9,-7.01,1.071624e+04,66.85
sum,3290689.00,3356663.0,-65974.00,1.007863e+08,628690.00
count,9405.00,9405.0,9405.00,9.405000e+03,9405.00


In [6]:
# Sum of all meters for each week
df2.drop(df2.columns[[2,3,4]], axis = 1, inplace = True)
df3 = df2.groupby(df2.index)['prediction','actual'].sum()
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['mse'] = round(df3['error'] ** 2,2)
df3['rmse'] = abs(df3['error'])
df3['%error'] = (df3['error']/df3['actual'])*100
print(df3)
df3.to_csv('CHL_SARIMA_Monthly_Persistence.csv')

    prediction    actual    error           mse     rmse     %error
0      81000.0   63843.0  17157.0  2.943626e+08  17157.0  26.873737
1      63843.0   57862.0   5981.0  3.577236e+07   5981.0  10.336663
2      57862.0   54026.0   3836.0  1.471490e+07   3836.0   7.100285
3      54026.0   73016.0 -18990.0  3.606201e+08  18990.0 -26.007998
4      73016.0   86413.0 -13397.0  1.794796e+08  13397.0 -15.503454
5      86413.0  113038.0 -26625.0  7.088906e+08  26625.0 -23.554026
6     113038.0  132271.0 -19233.0  3.699083e+08  19233.0 -14.540602
7     132271.0  136168.0  -3897.0  1.518661e+07   3897.0  -2.861906
8     136168.0  124168.0  12000.0  1.440000e+08  12000.0   9.664326
9     124168.0  123163.0   1005.0  1.010025e+06   1005.0   0.815992
10    123163.0  109928.0  13235.0  1.751652e+08  13235.0  12.039699
11    109928.0   86331.0  23597.0  5.568184e+08  23597.0  27.333171
12     86331.0   64729.0  21602.0  4.666464e+08  21602.0  33.372986
13     64729.0   60202.0   4527.0  2.049373e+07 

<ipython-input-6-a4839f81c055>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df2.groupby(df2.index)['prediction','actual'].sum()


In [7]:
# Summary of the sum of all meters for each week
df3.apply(Summary)

,prediction,actual,error,mse,rmse,%error
min,54026.00,54026.00,-36585.00,1.010025e+06,1005.00,-30.24
max,159650.00,159650.00,29314.00,1.338462e+09,36585.00,33.37
mean,99717.85,101717.06,-1999.21,3.273086e+08,15231.64,-0.09
sum,3290689.00,3356663.00,-65974.00,1.080118e+10,502644.00,-2.86
count,33.00,33.00,33.00,3.300000e+01,33.00,33.00
